In [48]:
import torch
import glob
import shutil
import os

print('Number of available devices ', torch.cuda.device_count())
# GPU 할당 변경하기

gpu_ids = 0  # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{gpu_ids}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
print('Current devices ', torch.cuda.current_device())

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, datasets, models
from loss_function import *
from dataload_eval import *
from utils import *
from metrics import *
import pandas as pd
# from model import *

# I expect to see RuntimeWarnings in this block

## 트레이닝 파라메터 설정하기
batch_size = 1
print(os.getcwd())
os.chdir('y:\Workspace\SM_KANG\workspace\liver_seg_cls')
data_dir =  './data/20220320_segmentation/220320_Dataset_raw'
ckpt_dir = './result/20220320_segmentation/checkpoint'

log_dir = './result/20220320_segmentation/log'
result_dir = './result/20220320_segmentation/result2'

mode = 'train'
train_continue = 'off'
kf_type = True

# device = 'cpu'
print("device: ", device)
print("batch size: %d" % batch_size)
print("data dir: %s" % data_dir)
print("ckpt dir: %s" % ckpt_dir)
print("log dir: %s" % log_dir)
print("result dir: %s" % result_dir)
print("mode: %s" % mode)
print("GPU ID : %s" % gpu_ids)
print("K-fold cross validation : %s" % str(kf_type))

## 디렉토리 생성하기
if not os.path.exists(result_dir):
    os.makedirs(os.path.join(result_dir, 'png'))
    os.makedirs(os.path.join(result_dir, 'numpy'))
    
def make_folder(pathL):
    for path in pathL:

        os.makedirs(os.path.join(result_dir, 'test', path))
    
def remove_folder(pathL):
    for path in pathL:
        if os.path.exists(os.path.join(result_dir, 'test', path)):
            shutil.rmtree(os.path.join(result_dir, 'test', path))

folderNames = [ 'Success_box_ns', 'Fail_box_ns',  'Success_box', 'Fail_box','Success', 'Fail','False_Positive','True_Negative','Crowd']
#remove_folder(folderNames)
#make_folder(folderNames)

## 그밖에 부수적인 functions 설정하기
fn_tonumpy = lambda x: x.to('cpu').detach().numpy().transpose(0, 2, 3, 1)
fn_denorm = lambda x, mean, std: (x * std) + mean
fn_class = lambda x: 1.0 * (x > 0.5)

## Tensorboard 를 사용하기 위한 SummaryWriter 설정
writer_train = SummaryWriter(log_dir=os.path.join(log_dir, 'train'))
writer_val = SummaryWriter(log_dir=os.path.join(log_dir, 'val'))
# early_stopping object의 초기화

# TEST MODE
# TEST MODE


from torchvision.models.segmentation.deeplabv3 import DeepLabHead
from PIL import Image
import scipy

Number of available devices  1
Current devices  0
y:\Workspace\SM_KANG\workspace\liver_seg_cls
device:  cuda:0
batch size: 1
data dir: ./data/20220320_segmentation/220320_Dataset_raw
ckpt dir: ./result/20220320_segmentation/checkpoint
log dir: ./result/20220320_segmentation/log
result dir: ./result/20220320_segmentation/result2
mode: train
GPU ID : 0
K-fold cross validation : True


In [49]:
from skimage import morphology
from scipy.signal import medfilt

def postprocessing(img,small_area):
    img = img / 255
    out1 = medfilt(img, 7)
    out3 = (morphology.remove_small_objects(out1.astype('bool'), small_area, connectivity=1) * 255).astype('uint8')
    
    return out3

In [50]:
def makeMask(img, w, h):
    maskpoint = np.array([[xx, yy] for xx, yy in zip(w, h)], dtype=np.int32)
    mask = cv2.fillPoly(img, [maskpoint], (255, 255, 255))

    return mask

In [51]:
from sklearn.metrics import jaccard_score

def IoU_cal(output, target):
    output_i = (output.reshape(output.shape[1] * output.shape[0]) / 255)
    target_i = (target.reshape(target.shape[1] * target.shape[0]) / 255)
    iou = round(jaccard_score(target_i, output_i, labels=np.unique(target_i), pos_label=1, average='binary'), 3)

    return iou

In [52]:
from torchvision import transforms, datasets, models

def draw_bbox(img,mask,size_re1,size_re2):
    if np.sum(mask) > 0:

        min_side = int(size_re1/8)
        y1 = np.min(np.where(mask == 255)[0]) 
        y2 = np.max(np.where(mask == 255)[0])
        x1 = np.min(np.where(mask == 255)[1]) 
        x2 = np.max(np.where(mask == 255)[1])

        box_fill = np.zeros((size_re1,size_re2))
        box_fill[y1-2:y2+1+2,x1-2:x2+1+2] = 255
        bbox = box_fill.copy()
        bbox[y1:y2+1, x1:x2+1] = 0

        img_bbox = img[y1:y2+1,x1:x2+1]
    else:
        box_fill = np.zeros((size_re1,size_re2))
        bbox = np.zeros((size_re1,size_re2))
        img_bbox = np.zeros((size_re1,size_re2))
    
    return box_fill, bbox, img_bbox

In [53]:
def draw_bbox_total(label, output,size1, size2):
    
    bbox_arr_label = np.zeros((size1, size2))
    bbox_arr_output = np.zeros((size1, size2))
    
    bbox_fill_label = bbox_arr_label.copy()
    bbox_fill_output = bbox_arr_output.copy()

    #zero_arr_total = zero_arr.copy()
    if np.sum(label) >0:
        #ret, imthres = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

        # 가장 바깥쪽 컨투어에 대해 모든 좌표 반환 ---③
        contour, _ = cv2.findContours(label, cv2.RETR_EXTERNAL, \
                                                         cv2.CHAIN_APPROX_NONE)
        N = len(contour)

        for i in range(0, N):
            xMax = np.max(contour[i][:,0,0])
            xMin = np.min(contour[i][:,0,0])
            yMax = np.max(contour[i][:,0,1])
            yMin = np.min(contour[i][:,0,1])
            
            bbox_arr_label[yMin-2:yMax+1+2, xMin-2:xMax+1+2] = 255
            bbox_fill_label = bbox_arr_label.copy()
            
            
            bbox_arr_label[yMin:yMax+1, xMin:xMax+1] = 0
            
    if np.sum(output) >0:
        #ret, imthres = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

        # 가장 바깥쪽 컨투어에 대해 모든 좌표 반환 ---③
        contour, _ = cv2.findContours(output, cv2.RETR_EXTERNAL, \
                                                         cv2.CHAIN_APPROX_NONE)
        N = len(contour)

        for i in range(0, N):
            xMax = np.max(contour[i][:,0,0])
            xMin = np.min(contour[i][:,0,0])
            yMax = np.max(contour[i][:,0,1])
            yMin = np.min(contour[i][:,0,1])
            
            bbox_arr_output[yMin-2:yMax+1+2, xMin-2:xMax+1+2] = 255
            bbox_fill_output = bbox_arr_output.copy()
            
            
            bbox_arr_output[yMin:yMax+1, xMin:xMax+1] = 0

    
    return  bbox_arr_label, bbox_arr_output, bbox_fill_label, bbox_fill_output


In [54]:
import cv2
import matplotlib.pyplot as plt
def multi_object(mask,size1,size2):
    
    img_list= []
    
    if np.sum(mask) >0:
        #ret, imthres = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

        # 가장 바깥쪽 컨투어에 대해 모든 좌표 반환 ---③
        contour, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, \
                                                         cv2.CHAIN_APPROX_NONE)
        N = len(contour)

        for i in range(0, N):
            img = np.zeros( (size1, size2) ) # create a single channel 200x200 pixel black image 
            img2= cv2.fillPoly(img, pts =[contour[i]], color=(255,255,255))
            img_list +=[img2]
    else:
        N =0
    
    return img_list, N

In [55]:
def draw_bbox2(input_i, label_i,output_i, size_re1,size_re2):
    label_box_fill, label_bbox, _ = draw_bbox(input_i, label_i,size_re1,size_re2 )
    output_box_fill, output_bbox, img_bbox = draw_bbox(input_i, output_i, size_re1,size_re2 )
    label_sum = label_bbox + label_i
    output_sum = output_bbox + output_i
    
    return label_sum, output_sum,label_box_fill,output_box_fill, img_bbox, output_bbox, label_bbox

In [56]:
def make_box(input_i,label_i, output_i, label_i_list,output_i_list, label_N ,output_N,size_re1,size_re2,  ID,result_dir,df_ps):
    ## Single object
    dice_arr =[]
    iou_arr = []
    img_box_arr = []
    id_arr =[]
    ID_sp = ID.split('.tiff')[0]
    label_bbox_total, output_bbox_total, bbox_fill_label_total, bbox_fill_output_total = draw_bbox_total(label_i, output_i,size_re1,size_re2)

    if label_N == 1 and output_N == 1:  ## True positive
        label_sum, output_sum,label_box_fill,output_box_fill, img_bbox, output_bbox, label_bbox = draw_bbox2(input_i,label_i_list[0],output_i_list[0], size_re1,size_re2 )
        Dice, IoU = make_metricNcaseImg(input_i,label_i, output_i, label_bbox_total, output_bbox_total, bbox_fill_label_total, bbox_fill_output_total, label_i_list[0],output_i_list[0],label_sum, output_sum,label_box_fill,output_box_fill,  img_bbox, ID,result_dir, size_re1,size_re2, output_bbox, label_bbox,df_ps)
        dice_arr +=[Dice]
        iou_arr +=[IoU]
        img_box_arr += [img_bbox]
        id_arr += [ID]

    elif label_N >= 1 and output_N == 0: ## False negative
        for i in range(0,label_N):
            label_sum, output_sum,label_box_fill,output_box_fill, img_bbox, output_bbox, label_bbox = draw_bbox2(input_i, label_i_list[i], np.zeros((size_re1,size_re2)), size_re1,size_re2)
            Dice, IoU = make_metricNcaseImg(input_i,label_i, output_i, label_bbox_total, output_bbox_total, bbox_fill_label_total, bbox_fill_output_total,  label_i_list[i], np.zeros((size_re1,size_re2)),label_sum, output_sum,label_box_fill,output_box_fill,  img_bbox,ID,result_dir, size_re1,size_re2, output_bbox, label_bbox,df_ps)
            dice_arr +=[Dice]
            iou_arr +=[IoU]
            ID = ID_sp.split('.tiff')[0] +str(i)+'.tiff'
            id_arr += [ID]
    
    ## Multi object
    else:
        IoU_L = []
        ii_L = []
        jj_L =[]
        count = 0

        for ii in range(0,label_N):
            label_box_fill, label_bbox, _ = draw_bbox(input_i, label_i_list[ii], size_re1,size_re2 )

            for jj in range(0, output_N):
                output_box_fill, output_bbox, img_bbox = draw_bbox(input_i, output_i_list[jj], size_re1,size_re2 )

                Dice, IoU = metrics_eval(output_box_fill/255, label_box_fill/255, size_re1,size_re2)
                IoU_L +=[IoU]
                ii_L += [ii]
                jj_L += [jj]
        df = pd.DataFrame(data ={'Label_n': ii_L, 'Pred_n': jj_L, 'IoU': IoU_L})
        Label_n = np.array(ii_L)
        Pred_n = np.array(jj_L)
        
        jj_L0 = np.unique(jj_L)
        output_count = output_N
        label_count = label_N
        pred_remain = np.arange(0,output_N,1)
        label_remain = np.arange(0, label_N,1)
        
        pred_uni = np.unique(list(df['Pred_n']))
        pred_uni_N = len(pred_uni)
        
        label_uni = np.unique(list(df['Label_n']))
        label_uni_N = len(label_uni)
            
        df2 = df.copy()
        
        pred_list = []
        label_list = []
        
        while len(df) > 0:

            idx = np.argmax(df['IoU'])
            
            print(df.iloc[idx]['Pred_n'])
           
            pred_idx = int(df.iloc[idx]['Pred_n'])
            label_idx = int(df.iloc[idx]['Label_n'] )
            
            pred_list +=[pred_idx]
            label_list +=[label_idx]
            
            count += 1
            ID =  ID_sp+str(count)+'.tiff'
                
            label_sum, output_sum,label_box_fill,output_box_fill, img_bbox, output_bbox, label_bbox = draw_bbox2(input_i, label_i_list[label_idx], output_i_list[pred_idx],size_re1,size_re2)
            Dice, IoU = make_metricNcaseImg(input_i,label_i, output_i, label_bbox_total, output_bbox_total, bbox_fill_label_total, bbox_fill_output_total, label_i_list[label_idx], output_i_list[pred_idx],label_sum, output_sum,label_box_fill,output_box_fill,  img_bbox, ID,result_dir, size_re1,size_re2,output_bbox, label_bbox,df_ps)
            dice_arr +=[Dice]
            iou_arr +=[IoU]
            img_box_arr += [img_bbox]
            id_arr += [ID]

            df = df[df['Pred_n'] != pred_idx]
            df = df[df['Label_n'] != label_idx]
            
        if label_uni_N > pred_uni_N:
            
            for i in range(0, len(label_list)):
                df2 = df2[(df2['Pred_n'] != pred_list[i]) | (df2['Label_n'] != label_list[i])]
                
            for i in range(0, len(df2)):
                label_idx = int(df2.iloc[i]['Label_n'])
                count += 1
                ID =  ID_sp+str(count)+'.tiff'
                             
                label_sum, output_sum,label_box_fill,output_box_fill, img_bbox, output_bbox, label_bbox = draw_bbox2(input_i,label_i_list[label_idx], np.zeros((size_re1,size_re2)),size_re1,size_re2)
                Dice, IoU = make_metricNcaseImg(input_i,label_i, output_i, label_bbox_total, output_bbox_total, bbox_fill_label_total, bbox_fill_output_total, label_i_list[label_idx], np.zeros((size_re1,size_re2)),label_sum, output_sum,label_box_fill,output_box_fill,  img_bbox, ID,result_dir, size_re1,size_re2,output_bbox, label_bbox,df_ps)
                dice_arr +=[Dice]
                iou_arr +=[IoU]
                img_box_arr += [img_bbox]
                id_arr += [ID]
                
        elif label_uni_N < pred_uni_N:
            
            for i in range(0, len(label_list)):
                df2 = df2[(df2['Pred_n'] != pred_list[i]) | (df2['Label_n'] != label_list[i])]
                
            for i in range(0, len(df2)):
                pred_idx = int(df2.iloc[i]['Pred_n'])
                count += 1
                ID =  ID_sp+str(count)+'.tiff'
                
                label_sum, output_sum,label_box_fill,output_box_fill, img_bbox, output_bbox, label_bbox = draw_bbox2(input_i, np.zeros((size_re1,size_re2)), output_i_list[pred_idx], size_re1,size_re2)
                Dice, IoU = make_metricNcaseImg(input_i,label_i, output_i, label_bbox_total, output_bbox_total, bbox_fill_label_total, bbox_fill_output_total,  np.zeros((size_re1,size_re2)), output_i_list[pred_idx], label_sum, output_sum,label_box_fill,output_box_fill,  img_bbox, ID,result_dir, size_re1,size_re2,output_bbox, label_bbox,df_ps)
                dice_arr +=[Dice]
                iou_arr +=[IoU]
                img_box_arr += [img_bbox]
                id_arr += [ID]

    return  dice_arr, iou_arr, img_box_arr,  id_arr

In [57]:
def make_metricNcaseImg(input_i, label_total, output_total, label_bbox_total, output_bbox_total, bbox_fill_label_total, bbox_fill_output_total, label_i, output_i, label_sum, output_sum,label_box_fill,output_box_fill,  img_bbox,ID,result_dir, size_re1,size_re2, output_bbox, label_bbox,df_ps):

    backtorgb = cv2.cvtColor(input_i, cv2.COLOR_GRAY2RGB)
    
    if np.sum(output_i) > 0:
        y1=int(np.min(np.where(output_bbox == 255)[0]))
        y2=int(np.max(np.where(output_bbox == 255)[0]))
        x1=int(np.min(np.where(output_bbox == 255)[1]))
        x2=int(np.max(np.where(output_bbox == 255)[1]))
        y_diff = y2-y1
        x_diff = x2-x1
        diff = abs((y_diff - x_diff)/2)            
        p1 = int(np.ceil(diff))
        p2 = int(np.floor(diff))
        
        if x_diff >= y_diff:
            y2_2 =y2+p2
            y1_2 =y1-p1
            x2_2 =x2
            x1_2 =x1   
            
        else:
            x2_2 =x2 + p2
            x1_2 =x1 - p1      
            y2_2 =y2
            y1_2 =y1  

        #backtorgb2 = backtorgb[y1:y2, x1:x2,:] 
        #output_i2 = output_i[y1:y2, x1:x2] 
        #backtorgb3 =  backtorgb2.copy()
        #backtorgb3[output_i2 == 0,:] = 0
        
        backtorgb2_2 = backtorgb[y1_2:y2_2, x1_2:x2_2,:] 
        output_i2_2 = output_i[y1_2:y2_2, x1_2:x2_2] 
        #backtorgb3_2 =  backtorgb2_2.copy()
        #backtorgb3_2[output_i2_2 == 0,:] = 0
        
    zero_box = np.zeros((size_re1, size_re2, 1))
    
    ###############################################################################

    label_rgb = np.concatenate(
        (label_i[:, :, np.newaxis], zero_box, zero_box),
        axis=2).astype('uint8')
    
    output_rgb = np.concatenate(
        (zero_box, zero_box, output_i[:, :, np.newaxis]),
        axis=2).astype('uint8')
    
    label_box_rgb = np.concatenate(
        ((label_bbox)[:, :, np.newaxis], zero_box, zero_box),
        axis=2).astype('uint8')
    
    output_box_rgb = np.concatenate(
        (zero_box, zero_box, (output_bbox)[:, :, np.newaxis]),
        axis=2).astype('uint8')
    
    backtorgb_box = backtorgb.copy()
    
    backtorgb_box[label_bbox == 255] = 0
    backtorgb_box[output_bbox == 255] = 0
    
    concat_img = (((backtorgb / 255) + (label_rgb / 255) + (output_rgb / 255)))

    concat_box = (((backtorgb_box / 255) + (label_box_rgb / 255) + (output_box_rgb / 255)))
    
    ###############################################################################
    #label_rgb_total = np.concatenate((label_total[:, :, np.newaxis], zero_box, zero_box),axis=2).astype('uint8')
    #print('output_total: ', np.shape(output_total))
    #print('output_bbox_total:', np.shape(output_bbox_total))
    #print('zero_box: ', np.shape(zero_box))
    
    #output_rgb_total = np.concatenate((zero_box, zero_box, output_total[:, :, np.newaxis]), axis=2).astype('uint8')
    #label_box_rgb_total = np.concatenate(((label_bbox_total)[:, :, np.newaxis], zero_box, zero_box),axis=2).astype('uint8')
    #output_box_rgb_total = np.concatenate((zero_box, zero_box, (output_bbox_total)[:, :, np.newaxis]), axis=2).astype('uint8')
    #backtorgb_total = backtorgb.copy()
    #backtorgb_box_total = backtorgb.copy()
    
    backtorgb_box_total[label_bbox_total == 255] = 0
    backtorgb_box_total[output_bbox_total == 255] = 0
    
    #concat_img_total = (((backtorgb_total / 255) + (label_rgb_total / 255) + (output_rgb_total / 255)))
    
    #concat_box_total = (((backtorgb_box_total / 255) + (label_box_rgb_total / 255) + (output_box_rgb_total / 255)))
    
    
    #######################################################################################################
    backtorgb_FP = backtorgb.copy()
    backtorgb_box_FP = backtorgb.copy()
    
    concat_img_FP = (((backtorgb_FP / 255) + (label_rgb_total / 255) + (output_rgb / 255)))
    
    
    concat_box_FP = (((backtorgb_box_FP / 255) + (label_box_rgb_total / 255) + (output_box_rgb / 255)))

    


    Dice, IoU = metrics_eval((output_box_fill/255), (label_box_fill/255), size_re1, size_re2)
    
    Dice_FP, IoU_FP = metrics_eval((output_box_fill/255), (bbox_fill_label_total/255), size_re1, size_re2)
    
    
    concat_img = cv2.resize(concat_img, (512,512), interpolation=cv2.INTER_CUBIC)
    concat_box = cv2.resize(concat_box, (512,512), interpolation=cv2.INTER_CUBIC)
    #concat_img_total = cv2.resize(concat_img_total, (512,512), interpolation=cv2.INTER_CUBIC)
    #concat_box_total = cv2.resize(concat_box_total, (512,512), interpolation=cv2.INTER_CUBIC)
    concat_img_FP = cv2.resize(concat_img_FP, (512,512), interpolation=cv2.INTER_CUBIC)
    concat_box_FP = cv2.resize(concat_box_FP, (512,512), interpolation=cv2.INTER_CUBIC)
    
    concat_img[concat_img > 1] = 1
    concat_box[concat_box > 1] = 1

    #concat_img_total[concat_img_total > 1] = 1
    #concat_box_total[concat_box_total > 1] = 1
    concat_img_FP[concat_img_FP > 1] = 1
    concat_box_FP[concat_box_FP > 1] = 1

    concat_img[concat_img < 0] = 0
    concat_box[concat_box < 0] = 0

    #concat_img_total[concat_img_total < 0] = 0
    #concat_box_total[concat_box_total < 0] = 0
    concat_img_FP[concat_img_FP < 0] = 0
    concat_box_FP[concat_box_FP < 0] = 0



        
    if (np.sum(output_i) > 0) and (IoU_FP == 0):  ## False positive
        IoU = -1
        Dice = -1 


    
    if np.sum(label_i)> 0:


            

        print('id: ',ID)
        print('IoU: ', IoU)
        



        if (IoU >= 0.20):

            plt.imsave(os.path.join(result_dir, 'test',  'Success', ID),concat_img)
            plt.imsave(os.path.join(result_dir, 'test',  'Success','box_'+ID),concat_box)

            
            #plt.imsave(os.path.join(result_dir, 'test',  'Success', 'Total_'+ID),concat_img_total)
            #plt.imsave(os.path.join(result_dir, 'test',  'Success','Total_box_'+ID),concat_box_total)


            
            plt.imsave(os.path.join(result_dir, 'test',  'Success_box',ID), backtorgb2_2)
            
            
        else:
            
            print('Fail')
            plt.imsave(os.path.join(result_dir, 'test',  'Fail', ID),concat_img)
            plt.imsave(os.path.join(result_dir, 'test',  'Fail','box_'+ID),concat_box)
            #plt.imsave(os.path.join(result_dir, 'test',  'Fail', 'Total_'+ID),concat_img_total)
            #plt.imsave(os.path.join(result_dir, 'test',  'Fail','Total_box_'+ID),concat_box_total)

  
            
        if (IoU == -1):
            plt.imsave(os.path.join(result_dir, 'test',  'False_Positive', ID),concat_img)
            plt.imsave(os.path.join(result_dir, 'test',  'False_Positive','box_'+ID),concat_box)
            #plt.imsave(os.path.join(result_dir, 'test',  'False_Positive', 'Total_'+ ID),concat_img_FP)
            #plt.imsave(os.path.join(result_dir, 'test',  'False_Positive','Total_box_'+ID),concat_box_FP)


    elif np.sum(label_i) ==0 and np.sum(output_i)> 0:
        if (IoU_FP == 0): ## 'False Positive'
            Dice= -1
            IoU = -1
            print('id: ',ID)
            print('False Positive')
            plt.imsave(os.path.join(result_dir, 'test',  'False_Positive', ID),concat_img)
            plt.imsave(os.path.join(result_dir, 'test',  'False_Positive','box_'+ID),concat_box)
            #plt.imsave(os.path.join(result_dir, 'test',  'False_Positive', 'Total_'+ ID),concat_img_FP)
            #plt.imsave(os.path.join(result_dir, 'test',  'False_Positive','Total_box_'+ID),concat_box_FP)

        else: ## 'Crowd'
            Dice= -2
            IoU = -2
            plt.imsave(os.path.join(result_dir, 'test',  'Crowd', 'Total_'+ ID),concat_img_FP)
            plt.imsave(os.path.join(result_dir, 'test',  'Crowd','Total_box_'+ID),concat_box_FP)


    elif np.sum(label_i) ==0 and np.sum(output_i)== 0: ## True Negative
            Dice= -3
            IoU = -3
            plt.imsave(os.path.join(result_dir, 'test',  'True_Negative', 'Total_'+ ID),concat_img_total)
            plt.imsave(os.path.join(result_dir, 'test',  'True_Negative','Total_box_'+ID),concat_box_total)


        
    print(Dice)
    print(IoU)

    
    return Dice, IoU

In [58]:
def save_best_npy(FOLD, testPath,df_ps):
    import matplotlib.pyplot as plt

    transform2 = transforms.Compose([Normalization(), ToTensor()])
    dataset_test = Dataset(data_dir=testPath, transform=transform2, target_transform=transform2)
    print(dataset_test )

    # 그밖에 부수적인 variables 설정하기
    num_data_test = len(dataset_test)
    print(num_data_test )

    ## 네트워크 생성하기
    deeplab = models.segmentation.deeplabv3_resnet101(pretrained=True)
    deeplab.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    deeplab.classifier = DeepLabHead(2048, 1)
    net = deeplab.to(device)

    ## Optimizer 설정하기
    optim = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9,  weight_decay=0.0001)

    loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, num_workers=0)
    
    print(loader_test )

    best_model_name = [j for j in os.listdir(ckpt_dir) if 'best_fold' + str(FOLD) in j][0]
    
    print((ckpt_dir))
    
    print(best_model_name )

    dict_model = torch.load('%s/%s' % (ckpt_dir, best_model_name),map_location = 'cuda:0')

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])
    
    #######################################################
    with torch.no_grad():
        print('step 1 --------------------------------')
        net.eval()

        count_total = 0

        dice_arr = []
        iou_arr = []
        fname_arr =[]
        
        dice_seg_arr = []
        iou_seg_arr = []
        fname_seg_arr =[]
        
        id_arr = []
        id_arr2  = []

        for batch, data in enumerate(loader_test, 1):
            print('step 2 --------------------------------')
            count = 0

            input_id = data['Input_ID']
            input_id1 = [i.split('/')[-1] for i in input_id]

            if data['Input_ID'][count].split('_img.')[0] != data['Label_ID'][count].split('_mask.')[0]:
                print('error!!!!!!!!!!!!!!!')
                break

            label = data['label'].to(device)
            input = data['input'].to(device)
            
            input_raw = data['input_raw']
            label_raw = data['label_raw']

            output = net(input)['out']
            sig = nn.Sigmoid()
            output = sig(output)

            # Tensorboard 저장하기
            label = fn_tonumpy(label)
            input = fn_tonumpy(input)
            output = fn_tonumpy(fn_class(output))
            
            for j in range(label.shape[0]):
                ID= data['Input_ID'][j]
                print('ID: ', ID)

                #print('small area: ', small_area)
                
                #input_i = (input[j].squeeze() * 255).astype('uint8')
                #label_i = (label[j].squeeze() * 255).astype('uint8')
                #output_i = (output[j].squeeze() * 255).astype('uint8')
                #label_i =cv2.resize(label, (input_raw.shape[0], input_raw.shape[1])
                output_i = output[j][:,:,0]

                input_i = input_raw[j][:,:,0].numpy()
                label_i = label_raw[j][:,:,0].numpy()
                print('label_i.shape: ', label_i.shape)
                print('output_i.shape: ', output_i.shape)
                output_i =cv2.resize(output_i, (label_i.shape[1], label_i.shape[0]), interpolation=cv2.INTER_NEAREST)
                
                img_fid = ID.split('_class')[0]
                ps = df_ps[df_ps['fid'] == img_fid ]['ps'].values
 
                small_area = int(0.25/(ps*ps))
    
                output_i = postprocessing(output_i,small_area)
                
                Dice_seg, IoU_seg = metrics_eval((output_i/255).astype('int'), (label_i/255).astype('int'),label_i.shape[1], label_i.shape[0])
            
                dice_seg_arr += [Dice_seg]
                iou_seg_arr += [IoU_seg]
 
                print('label_i.shape: ', label_i.shape)
                print('output_i.shape: ', output_i.shape)
                
                print('label_i.max: ', np.max(label_i))
                print('output_i.max: ', np.max(output_i))
                
                size_re1 = label_i.shape[0]
                size_re2 = label_i.shape[1]
                
                label_i_list, label_N=multi_object(label_i ,size_re1,size_re2)
                output_i_list, output_N=multi_object(output_i ,size_re1,size_re2)
                            
                Dice, IoU,imgBox, idArr = make_box(input_i,label_i, output_i, label_i_list,output_i_list, label_N ,output_N,size_re1,size_re2,  ID ,result_dir,df_ps)
                dice_arr +=Dice
                iou_arr +=IoU
                fname_arr += [ID]
                
                print('len(dice_seg_arr): ', len(dice_seg_arr))
                print('len(dice_seg_arr): ', len(dice_seg_arr))
                print('len(iou_seg_arr): ', len(iou_seg_arr))
                
                types_num = ID.split('class_')[-1].split('_')[0]
                if types_num == '0':
                    types = 'HCC'
                elif types_num == '1':
                    types = 'Hemangioma'
                elif types_num == '2':
                    types = 'Cyst'
                id_arr2 +=[types]

                id_arr  += [types ]*len(Dice)
                count += 1
                count_total += 1

        results = pd.DataFrame(data={ 'IoU_detect': iou_arr,  'Type': id_arr})
        results.to_csv(os.path.join(result_dir, 'test', 'Results_Detection_all_' + str(FOLD) + '.csv'))
        
        results_seg = pd.DataFrame(data={'ID':fname_arr,'IoU_seg': iou_seg_arr,'DSC_seg': dice_seg_arr,  'Type': id_arr2})
        results_seg.to_csv(os.path.join(result_dir, 'test', 'Results_Segmentation_all_' + str(FOLD) + '.csv'))
        
        dsc_all_seg = np.mean(results_seg['DSC_seg'])
        iou_all_seg = np.mean(results_seg['IoU_seg'])
        df_seg = pd.DataFrame(data={'iou_all_seg': iou_all_seg,'dsc_all_seg': dsc_all_seg}, index=[0])

        iou_all = np.mean(results['IoU_detect'])
        df_detect1 = pd.DataFrame(data={'iou_all': iou_all}, index=[0])
        
        df_seg.to_csv(os.path.join(result_dir, 'test', 'Results_Segmentation' + str(FOLD) + '.csv'))
        df_detect1.to_csv(os.path.join(result_dir, 'test', 'Results_Detection' + str(FOLD) + '.csv'))

In [59]:
def get_cv(fold, data_dir):

    cv_num = [0,1,2,3,4]
    train = []

    for i in range(0,5):
        num = fold+i
        if num >=5:
            num = num-5
        if i <= 2:
            train_i = os.path.join(data_dir, 'CV'+str(cv_num[num]+1))
            train += [train_i]
        if i ==3:
            val = [os.path.join(data_dir, 'CV'+str(cv_num[num]+1))]
        if i == 4:
            test =  [os.path.join(data_dir, 'CV'+str(cv_num[num]+1))]

    return train, val, test

In [60]:
path = './data/PS_info.csv'
df_ps = pd.read_csv(path)

for fold in range(0,5):
    FOLD = fold+1
    trainPath, valPath, testPath = get_cv(fold, data_dir)
    save_best_npy(FOLD, testPath,df_ps)

---------------------------
0


IndexError: list index out of range

In [ ]:
df1 = pd.read_csv(path1)
df1_TP_FN = df1[df1['IoU_detect'] >= -1]
df1_FP = df1[df1['IoU_detect'] < 0]
df1_box_IoU = df1['IoU_detect'].replace([-1,-2],[0,0])

df2 = pd.read_csv(path2)
df2_TP_FN = df2[df2['IoU_detect'] >=-1]
df2_FP = df2[df2['IoU_detect'] < 0]
df2_box_IoU = df2['IoU_detect'].replace([-1,-2],[0,0])

df3 = pd.read_csv(path3)
df3_TP_FN = df3[df3['IoU_detect'] >=-1]
df3_FP = df3[df3['IoU_detect'] < 0]
df3_box_IoU = df3['IoU_detect'].replace([-1,-2],[0,0])

df4 = pd.read_csv(path4)
df4_TP_FN = df4[df4['IoU_detect'] >=-1]
df4_FP = df4[df4['IoU_detect'] < 0]
df4_box_IoU = df4['IoU_detect'].replace([-1,-2],[0,0])

df5 = pd.read_csv(path5)
df5_TP_FN = df5[df5['IoU_detect'] >= -1]
df5_FP = df5[df5['IoU_detect'] < 0]
df5_box_IoU = df5['IoU_detect'].replace([-1,-2],[0,0])

df_all = pd.concat([df1,df2,df3,df4,df5],axis = 0)
df_all_TP_FN = df_all[df_all['IoU_detect'] >= -1]
df_all_FP = df_all[df_all['IoU_detect'] < 0]
df_all_box_IoU = df_all['IoU_detect'].replace([-1,-2],[0,0])

In [ ]:
from math import sqrt

def ci(score, y_true, positive=1):
    #y_true = np.array((y_true)).astype('int')
    #AUC = roc_auc_score(y_true, y_score,multi_class='ovr')

    
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = score / (2 - score)
    Q2 = 2*score**2 / (1 + score)
    SE_score = sqrt((score*(1 - score) + (N1 - 1)*(Q1 - score**2) + (N2 - 1)*(Q2 - score**2)) / (N1*N2))
    lower = score - 1.96*SE_score
    upper = score + 1.96*SE_score
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return lower, upper

In [ ]:
def cal_detection_rate(df, L, CV):
    DC = df['IoU_detect'][df['IoU_detect'] > 0].copy()
    DC[DC >= 0.2] = 1
    DC[DC< 0.2] = 0
    DR= np.round(len(DC[DC>= 0.2])/ L,3)
    lower, upper = ci(DR,DC)
    print('Detection rate of '+CV+' : %.3f(%.3f-%.3f)' %(DR, lower,  upper))

In [ ]:
def cal_FP_rate(df, L, CV):
    
    FC = df['IoU_detect'].copy()
    FC[FC >= 0] = 0
    FC[FC < 0] = 1
    

    FR= np.round(len(df[df['IoU_detect']<0])/ L,3)

    lower, upper = ci(FR,FC)
    print('False Positive Rate of '+CV+' : %.3f(%.3f-%.3f)' %(FR, lower,  upper))

In [ ]:
cal_detection_rate(df1_TP_FN, 1468, 'CV1')
cal_detection_rate(df2_TP_FN, 1123, 'CV2')
cal_detection_rate(df3_TP_FN, 1294, 'CV3')
cal_detection_rate(df4_TP_FN, 1503, 'CV4')
cal_detection_rate(df5_TP_FN, 1434, 'CV5')
cal_detection_rate(df_all_TP_FN, (1123+1294+1503+1434+1468), 'all')

In [ ]:
print('False Positive Rate')

cal_FP_rate(df1,1437, 'CV1')
cal_FP_rate(df2,1104, 'CV2')
cal_FP_rate(df3,1275, 'CV3')
cal_FP_rate(df4,1479, 'CV4')
cal_FP_rate(df5,1417, 'CV5')
cal_FP_rate(df_all,(1437+1104+1275+1479+1417), 'all')

In [ ]:
path1 = './result/20220320_segmentation/result/test/Results_Segmentation_all_1.csv'
path2 = './result/20220320_segmentation/result/test/Results_Segmentation_all_2.csv'
path3 = './result/20220320_segmentation/result/test/Results_Segmentation_all_3.csv'
path4 = './result/20220320_segmentation/result/test/Results_Segmentation_all_4.csv'
path5 = './result/20220320_segmentation/result/test/Results_Segmentation_all_5.csv'
df1 = pd.read_csv(path1)

In [ ]:
df1 = pd.read_csv(path1)
df1_IoU = df1['IoU_seg'] 
df1_DSC = df1['DSC_seg'] 

df2 = pd.read_csv(path2)
df2_IoU = df2['IoU_seg'  ] 
df2_DSC = df2['DSC_seg'] 

df3 = pd.read_csv(path3)
df3_IoU = df3['IoU_seg'] 
df3_DSC = df3['DSC_seg'] 

df4 = pd.read_csv(path4)
df4_IoU = df4['IoU_seg'] 
df4_DSC = df4['DSC_seg'] 

df5 = pd.read_csv(path5)
df5_IoU = df5['IoU_seg'] 
df5_DSC = df5['DSC_seg'] 

df_all = pd.concat([df1,df2,df3,df4,df5],axis = 0)
df_all_IoU = df_all['IoU_seg'] 
df_all_DSC = df_all['DSC_seg'] 

In [ ]:
df_all.to_csv('./result/20220320_segmentation/result/test/Results_Segmentation_all_total.csv',index=None)

In [ ]:
print(np.round(np.mean(df1_IoU),3))
print(np.round(np.std(df1_IoU),3))
print('-'*30)
print(np.round(np.mean(df2_IoU),3))
print(np.round(np.std(df2_IoU),3))
print('-'*30)
print(np.round(np.mean(df3_IoU),3))
print(np.round(np.std(df3_IoU),3))
print('-'*30)
print(np.round(np.mean(df4_IoU),3))
print(np.round(np.std(df4_IoU),3))
print('-'*30)
print(np.round(np.mean(df5_IoU),3))
print(np.round(np.std(df5_IoU),3))
print('-'*30)
print(np.round(np.mean(df_all_IoU),3))
print(np.round(np.std(df_all_IoU),3))
print('-'*30)

In [ ]:
print(np.round(np.mean(df1_DSC),3))
print(np.round(np.std(df1_DSC),3))
print('-'*30)
print(np.round(np.mean(df2_DSC),3))
print(np.round(np.std(df2_DSC),3))
print('-'*30)
print(np.round(np.mean(df3_DSC),3))
print(np.round(np.std(df3_DSC),3))
print('-'*30)
print(np.round(np.mean(df4_DSC),3))
print(np.round(np.std(df4_DSC),3))
print('-'*30)
print(np.round(np.mean(df5_DSC),3))
print(np.round(np.std(df5_DSC),3))
print('-'*30)
print(np.round(np.mean(df_all_DSC),3))
print(np.round(np.std(df_all_DSC),3))
print('-'*30)

In [ ]:
os.chdir('/home/ubuntu/gcubme_ai/Workspace/S_Oh/US_Liver')
